# Visualising GEE Data
This notebook pulls varying bands from the HYCOM Dataset and outputs as .PNG files for inclusion into the document

In [1]:
import ee

ee.Initialize()
ee.Authenticate()

Enter verification code:  4/1AX4XfWjTbBJrE7n27HHRCizJfNXwWyvFmntrlCf8fiDK--l2vrYCPTMmLgc



Successfully saved authorization token.


In [6]:
import folium
from folium import plugins
import colorcet as cc

In [9]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [7]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except Exception as e:
        print(f"{e}: Could not display {name}")
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [27]:
dataset = ee.ImageCollection('HYCOM/sea_temp_salinity').filter(ee.Filter.date('2019-06-01', '2019-07-01'))


def map_to_c(image):
    return ee.Image(image).multiply(0.001).add(20)

dataset = dataset.map(lambda image: map_to_c(image))
dataset = dataset.select('water_temp_0')
vis_params = {
    'min': 20,
    'max': 34,
    'palette': cc.fire,
}

# Create a folium map object.
my_map = folium.Map(location=[-112.85, 36], zoom_start=9)

# Add custom basemaps
# basemaps['Google Maps'].add_to(my_map)
# basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(dataset, vis_params, 'test')
# Display the map.
display(my_map)

In [35]:
CONST = (34-20)/256
# creating legend to overlay
vals = [i*CONST for i in range(0, 257)]
legend =[(cc.fire[i], vals[i]+20) for i in [0,50,100,150,200,250]]
legend

[('#000000', 20.0),
 ('#660100', 22.734375),
 ('#bb0600', 25.46875),
 ('#fd5100', 28.203125),
 ('#ffb402', 30.9375),
 ('#fffea8', 33.671875)]

In [46]:
dataset = ee.ImageCollection('HYCOM/sea_temp_salinity').filter(ee.Filter.date('2019-06-01', '2019-07-01'))


def map_to_c(image):
    return ee.Image(image).multiply(0.001).add(20)

dataset = dataset.map(lambda image: map_to_c(image))
dataset = dataset.select('salinity_0')

vis_params = {
    'min': 30,
    'max': 40,
    'palette': cc.kbc,
}

# Create a folium map object.
my_map = folium.Map(location=[-112.85, 36], zoom_start=9)

# Add custom basemaps
# basemaps['Google Maps'].add_to(my_map)
# basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(dataset, vis_params, 'test')
# Display the map.
display(my_map)

In [49]:
CONST = (30-40)/256
# creating legend to overlay
vals = [i*CONST for i in range(0, 257)]
legend =[(cc.kbc[i], vals[i]+20) for i in [0,50,100,150,200,250]]
legend

[('#00004e', 20.0),
 ('#0905b9', 18.046875),
 ('#2d42fa', 16.09375),
 ('#2e8afd', 14.140625),
 ('#28c8fb', 12.1875),
 ('#a3faf6', 10.234375)]